# Segmenting and Clustering Neighborhoods in Toronto

### Read data from webpage

In [25]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

#Read data from Wikipedia page
toronto_rawdata = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

#assign the table data to a new data frame
dfToronto = toronto_rawdata[0]

#Assignment requires three columns: PostalCode, Borough, and Neighborhood. Rename the column "Postcal Code" to "PostcalCode"
dfToronto.rename(columns = {'Postal Code':'PostalCode'}, inplace = True) 

#dfToronto

#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
#Filter out the not assigned borough
dfToronto_AssignedBorough = dfToronto[dfToronto['Borough']!='Not assigned']

#merge Neighbourhood with same PostcalCode
dtToronto_Merged = dfToronto_AssignedBorough.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
#CHeck if there is any not assigned neighborhoud, The merge happened first, a not-assigned neighborhood could be merged
dtToronto_Merged.loc[(dtToronto_Merged['Neighbourhood'].str.contains('Not assigned')),'Neighbourhood'] = dtToronto_Merged.loc[(dtToronto_Merged['Neighbourhood'].str.contains('Not assigned')),'Borough']


In [27]:
#use the .shape method to print the number of rows of your dataframe.
dtToronto_Merged.shape

(103, 3)

In [34]:
#Install the geocoder package
!pip install geocoder

     |████████████████████████████████| 98 kB 6.8 MB/s eta 0:00:011


In [37]:
import geocoder # import geocoder

#********************************************* 
#Give a try, the geocoder doesn't work
#********************************************* 

def get_geocoder(postal_code_from_df):
     # initialize your variable to None
     lat_lng_coords = None
     # loop until you get the coordinates
     while(lat_lng_coords is None):
       g = geocoder.google('{}, Toronto, Ontario'.format(postal_code_from_df))
       lat_lng_coords = g.latlng
     latitude = lat_lng_coords[0]
     longitude = lat_lng_coords[1]
     return latitude,longitude

#testlat,testlong = get_geocoder('M5G')
print('Geocoder doesnot work, load from csv')

#for i in range(0,len(dtToronto_Merged)):
#    dtToronto_Merged['Latitude'][i],dtToronto_Merged['Longitude'][i]=get_geocoder(dtToronto_Merged.iloc[i]['PostalCode'])


Geocoder doesnot work, load from csv


In [38]:
#Read CSV file to get Lat/Long from Postal Code
dfPostalCode2LatLong = pd.read_csv('https://cocl.us/Geospatial_data')

In [41]:
#Merge using Inner Join
dfToronto = pd.merge(left=dtToronto_Merged, right=dfPostalCode2LatLong, left_on='PostalCode', right_on='Postal Code')

dfToronto.drop('Postal Code', axis=1, inplace=True)

#Size of new data.
dfToronto.shape
dfToronto


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
